<a href="https://colab.research.google.com/github/ghr8635/E2E-DriveAI-ROS2-based-Modular-Framework-for-Autonomous-Vehicle-Control/blob/main/self_built_point_pillar_architecture_(simple_data).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset, DataLoader

**Define the PointPillars Feature Extraction Model**

In [ ]:
class PillarFeatureEncoder(nn.Module):
    def __init__(self, in_channels, out_channels, num_points_per_pillar, grid_x, grid_y):
        super(PillarFeatureEncoder, self).__init__()
        self.num_points_per_pillar = num_points_per_pillar
        self.grid_x, self.grid_y = grid_x, grid_y
        # Simple linear layer to encode pillar features
        self.fc = nn.Linear(in_channels, out_channels)

    def forward(self, pillars):
        # Flatten and encode features
        pillars = self.fc(pillars)  # Shape: (batch, grid_x, grid_y, num_points, out_channels)
        return pillars.mean(dim=2)  # Reduce across point dimension to get (batch, grid_x, grid_y, out_channels)

class BackboneNetwork(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(BackboneNetwork, self).__init__()
        # Simple 2D CNN layers for feature extraction
        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(128, out_channels, kernel_size=3, stride=2, padding=1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.conv3(x)  # Shape: (batch, out_channels, grid_x//4, grid_y//4)
        return x

class PointPillarsFeatureExtractor(nn.Module):
    def __init__(self, in_channels=4, out_channels=256, num_points_per_pillar=32, grid_x=100, grid_y=100):
        super(PointPillarsFeatureExtractor, self).__init__()
        self.pfe = PillarFeatureEncoder(in_channels, out_channels, num_points_per_pillar, grid_x, grid_y)
        self.backbone = BackboneNetwork(out_channels, out_channels)

    def forward(self, x):
        pillar_features = self.pfe(x)  # Shape: (batch, grid_x, grid_y, out_channels)
        pillar_features = pillar_features.permute(0, 3, 1, 2)  # Shape: (batch, out_channels, grid_x, grid_y)
        feature_map = self.backbone(pillar_features)
        return feature_map

**Prepare a Simple Dataset for Training**

In [ ]:
class SyntheticPointCloudDataset(Dataset):
    def __init__(self, num_samples, grid_x=100, grid_y=100, num_points_per_pillar=32, in_channels=4):
        self.num_samples = num_samples
        self.grid_x, self.grid_y = grid_x, grid_y
        self.num_points_per_pillar = num_points_per_pillar
        self.in_channels = in_channels

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        # Generate random pillars with shape: (grid_x, grid_y, num_points, in_channels)
        pillars = np.random.rand(self.grid_x, self.grid_y, self.num_points_per_pillar, self.in_channels).astype(np.float32)
        return torch.tensor(pillars)

# Parameters
num_samples = 100
dataset = SyntheticPointCloudDataset(num_samples)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)


**Training Loop (Optional, to learn feature extraction patterns)**

In [ ]:
# Instantiate model, optimizer, and criterion
model = PointPillarsFeatureExtractor()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Simple Training Loop
epochs = 5
for epoch in range(epochs):
    for batch_idx, pillars in enumerate(dataloader):
        optimizer.zero_grad()

        # Forward pass
        features = model(pillars)  # Extracted features, shape: (batch, out_channels, grid_x//4, grid_y//4)

        # Dummy target: here we're using the output itself as target for demonstration
        target = features.clone().detach()  # This is just for illustrative purposes
        loss = criterion(features, target)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        print(f"Epoch [{epoch+1}/{epochs}], Batch [{batch_idx+1}/{len(dataloader)}], Loss: {loss.item():.4f}")


Epoch [1/5], Batch [1/25], Loss: 0.0000
Epoch [1/5], Batch [2/25], Loss: 0.0000
Epoch [1/5], Batch [3/25], Loss: 0.0000
Epoch [1/5], Batch [4/25], Loss: 0.0000
Epoch [1/5], Batch [5/25], Loss: 0.0000
Epoch [1/5], Batch [6/25], Loss: 0.0000
Epoch [1/5], Batch [7/25], Loss: 0.0000
Epoch [1/5], Batch [8/25], Loss: 0.0000
Epoch [1/5], Batch [9/25], Loss: 0.0000
Epoch [1/5], Batch [10/25], Loss: 0.0000
Epoch [1/5], Batch [11/25], Loss: 0.0000
Epoch [1/5], Batch [12/25], Loss: 0.0000
Epoch [1/5], Batch [13/25], Loss: 0.0000
Epoch [1/5], Batch [14/25], Loss: 0.0000
Epoch [1/5], Batch [15/25], Loss: 0.0000
Epoch [1/5], Batch [16/25], Loss: 0.0000
Epoch [1/5], Batch [17/25], Loss: 0.0000
Epoch [1/5], Batch [18/25], Loss: 0.0000
Epoch [1/5], Batch [19/25], Loss: 0.0000
Epoch [1/5], Batch [20/25], Loss: 0.0000
Epoch [1/5], Batch [21/25], Loss: 0.0000
Epoch [1/5], Batch [22/25], Loss: 0.0000
Epoch [1/5], Batch [23/25], Loss: 0.0000
Epoch [1/5], Batch [24/25], Loss: 0.0000
Epoch [1/5], Batch [25/25

**Saving Model**

In [ ]:
# Define the file path where you want to save the model
model_save_path = "/content/drive/MyDrive/ROS2-Modular-Framework-for-End-to-End-Autonomous-Vehicle-Control-from-Raw-Sensor-Data/self_built_point_pillar.pth"

# After training is complete, save the model
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")


Model saved to /content/drive/MyDrive/ROS2-Modular-Framework-for-End-to-End-Autonomous-Vehicle-Control-from-Raw-Sensor-Data/self_built_point_pillar.pth


**Loading the Model for Inference**

In [ ]:
# Initialize the model architecture
loaded_model = PointPillarsFeatureExtractor()

# Load the saved weights
loaded_model.load_state_dict(torch.load(model_save_path))

# Set the model to evaluation mode if you are not training it further
loaded_model.eval()

print("Model loaded successfully.")


Model loaded successfully.


<ipython-input-9-2d3a27d5251c>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model.load_state_dict(torch.load(model_save_path))
